In [1]:
from data_loader import load_data

In [24]:
import numpy as np
import random

# PSO Algorithm
def pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs):
    # PSO Parameters
    w = 0.9  # Inertia weight
    c1 = 1.5  # Personal learning coefficient
    c2 = 1.5  # Social learning coefficient
    n_particles = 50  # Number of particles
    n_iterations = 100  # Number of iterations

    # Initialize particles randomly (each particle represents a possible solution)
    particles = np.array([np.random.randint(0, n_depots, n_customers) for _ in range(n_particles)])  # Random solutions
    velocities = np.zeros_like(particles)  # Initial velocities (0)

    # Initializing personal best and global best solutions
    personal_best = np.copy(particles)
    personal_best_costs = np.array([calculate_cost(solution, n_customers, n_depots, depot_capacities, setup_costs, customer_demands, costs) for solution in particles])

    global_best = personal_best[np.argmin(personal_best_costs)]
    global_best_cost = np.min(personal_best_costs)

    for iteration in range(n_iterations):
        for i in range(n_particles):
            # Velocity update (particle movement)
            velocities[i] = (w * velocities[i] +
                             c1 * random.random() * (personal_best[i] - particles[i]) +
                             c2 * random.random() * (global_best - particles[i]))

            # Position update (new solutions)
            particles[i] = particles[i] + velocities[i]
            
            # Ensure the solution is within the valid range (between 0 and n_depots-1)
            particles[i] = np.clip(particles[i], 0, n_depots - 1)

            # Ensure depot capacity is not exceeded
            depot_usage = np.zeros(n_depots)
            for customer in range(n_customers):
                depot = int(particles[i][customer])  # Which depot the customer is assigned to
                depot_usage[depot] += customer_demands[customer]  # Sum of demands for the assigned depot
            
            # Penalize solutions where depot capacity is exceeded
            penalty = 0
            for depot in range(n_depots):
                if depot_usage[depot] > depot_capacities[depot]:
                    penalty += 100000  # Apply large penalty for exceeding capacity

            # Calculate the cost for the new solution with the penalty
            cost = calculate_cost(particles[i], n_customers, n_depots, depot_capacities, setup_costs, customer_demands, costs) + penalty

            # Update personal best if new solution is better
            if cost < personal_best_costs[i]:
                personal_best[i] = particles[i]
                personal_best_costs[i] = cost

        # Update global best if personal best is better
        min_cost_idx = np.argmin(personal_best_costs)
        if personal_best_costs[min_cost_idx] < global_best_cost:
            global_best = personal_best[min_cost_idx]
            global_best_cost = personal_best_costs[min_cost_idx]

        # Decrease inertia weight `w` over time
        w = 0.9 - (0.5 * iteration / n_iterations)  # Decrease inertia weight

    return global_best, global_best_cost

# Cost Calculation Function
def calculate_cost(solution, n_customers, n_depots, depot_capacities, setup_costs, customer_demands, costs):
    total_cost = 0
    depot_usage = np.zeros(n_depots)

    for customer in range(n_customers):
        depot = int(solution[customer])  # Which depot is assigned to the customer
        depot_usage[depot] += customer_demands[customer]  # Track customer demands per depot

    # Calculate setup costs and transportation costs
    for depot in range(n_depots):
        if depot_usage[depot] > 0:
            total_cost += setup_costs[depot]  # Setup cost for the depot
            for customer in range(n_customers):
                if int(solution[customer]) == depot:
                    total_cost += costs[depot, customer]  # Transportation cost from depot to customer

    return total_cost


In [25]:
# Veri setinizi yükleyin
filename = 'Dataset/wl_25'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

n_depots = np.array(n_depots)  # Depo sayısı
n_customers = np.array(n_customers)  # Müşteri sayısı
depot_capacities = np.array(depot_capacities)  # Depo kapasiteleri
setup_costs = np.array(setup_costs)  # Depo kurulum maliyetleri
customer_demands = np.array(customer_demands)  # Müşteri talepleri
costs = np.array(costs)  # Müşteri-depo maliyetleri


# Depo ve müşteri sayısını tekrar belirleyelim
n_depots = len(depot_capacities)  # Depo sayısı
n_customers = len(customer_demands)  # Müşteri sayısı

# costs matrisini numpy dizisine çevir
costs = np.array(costs)  # Müşteri-depo maliyetleri

# costs matrisinin boyutlarını kontrol et
print(f"Mevcut costs boyutu: {costs.shape}")
print(f"Beklenen costs boyutu: ({n_depots}, {n_customers})")

# costs matrisinin boyutları uyumsuzsa, uygun şekilde düzeltelim
if costs.shape[0] != n_depots or costs.shape[1] != n_customers:
    print("costs matrisinin boyutları uyumsuz!")
    print(f"Mevcut costs boyutu: {costs.shape}")
    print(f"Beklenen costs boyutu: ({n_depots}, {n_customers})")

    # costs matrisinin boyutlarını transpoze edelim
    if costs.shape[0] == n_customers and costs.shape[1] == n_depots:
        costs = costs.T  # Eğer satırlar ve sütunlar yer değişmişse, transpoz alıyoruz
        print("costs matrisini transpoze ettik.")
    elif costs.shape[0] != n_depots or costs.shape[1] != n_customers:
        print("costs matrisinin boyutları hala uyumsuz!")
        costs = costs[:n_depots, :n_customers]
        print("costs matrisini uygun boyutlarda kestik.")
    
    print(f"Yeni costs boyutu: {costs.shape}")
else:
    print("costs matrisinin boyutları uyumlu.")

# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)

Mevcut costs boyutu: (50, 25)
Beklenen costs boyutu: (25, 50)
costs matrisinin boyutları uyumsuz!
Mevcut costs boyutu: (50, 25)
Beklenen costs boyutu: (25, 50)
costs matrisini transpoze ettik.
Yeni costs boyutu: (25, 50)
En iyi çözüm: [18 18  8 24 24 19 16 24 18 21  3  4 20  0 13 16  7 18 19  8 19 21 19 24
 11  0 21 18  7 17  0  4 19  1  7 18 17  4 21  8 19  7  7  9 21  4  0  9
 18 11]
En düşük maliyet: 1285478.75


In [26]:
filename = 'Dataset/wl_50'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

n_depots = np.array(n_depots)  # Depo sayısı
n_customers = np.array(n_customers)  # Müşteri sayısı
depot_capacities = np.array(depot_capacities)  # Depo kapasiteleri
setup_costs = np.array(setup_costs)  # Depo kurulum maliyetleri
customer_demands = np.array(customer_demands)  # Müşteri talepleri
costs = np.array(costs)  # Müşteri-depo maliyetleri


# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)

En iyi çözüm: [ 0  1 49  0 49  0  1  1 49  0 49  0  0  0  0  0 49  0 49  0  0 49 49 49
  1  0 49  0 49  1  0  0 49 49 49  0 49  0 49  0  0 49 49 49  0  0  0  0
  0  0]
En düşük maliyet: 578082.2124999999


In [27]:
filename = 'Dataset/wl_200'
n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs = load_data(filename)

n_depots = np.array(n_depots)  # Depo sayısı
n_customers = np.array(n_customers)  # Müşteri sayısı
depot_capacities = np.array(depot_capacities)  # Depo kapasiteleri
setup_costs = np.array(setup_costs)  # Depo kurulum maliyetleri
customer_demands = np.array(customer_demands)  # Müşteri talepleri
costs = np.array(costs)  # Müşteri-depo maliyetleri


# PSO ile çözümü iyileştiriyoruz
best_solution, best_cost = pso_algorithm(n_depots, n_customers, depot_capacities, setup_costs, customer_demands, costs)

# Sonuçları yazdır
print("En iyi çözüm:", best_solution)
print("En düşük maliyet:", best_cost)

En iyi çözüm: [199 199   0   0 199   0 199   0   0 199 199   0   0   0   0   0   0   0
 199 199   0 199   0 199 199 199 199   0   0 199 199 199 199   0   0 199
 199   0 199   0   0 199   0 199   0   0 199   0 199   0   0 199   0   0
 199 199   0 199   0 199   0 199   0   0   0 199   0 199   0   0   0 199
   0 199 199   0   0   0   0 199   0   0   0 199 199 199   0   0 199 199
 199   0   0 199   0 199   0   0   0   0 199 199   0   0 199   0   0   0
   0 199 199   0   0 199 199   0   0 199   0   0 199 199   0   0   0 199
 199 199 199   0   0   0 199 199   0   0 199 199 199 199 199 199 199 199
   0   0 199   0 199 199 199 199   0 199   0   0 199   0 199 199   0 199
 199   0   0   0 199 199 199 199 199 199 199 199 199 199 199 199 199 199
   0   0   0   0   0   0   0 199   0 199   0   0 199 199   0   0 199   0
   0   0]
En düşük maliyet: 3080.3944800000027
